# 盘中动态:20241119

In [1]:
import nbimporter
from 盘前作业20241119 import plotStock

repday='20241119'

## 1. 今日板块热点


![今日涨幅国泰君安APP](gtja.png)

定时任务运行器：集合竞价和开盘前30分钟

In [2]:
import akshare as ak

symbol='600839'
# 获取股票数据
df = ak.stock_bid_ask_em(symbol=symbol)

# 转换为字典
data_dict = df.set_index('item').to_dict()['value']

In [3]:
data_dict

{'sell_5': 11.95,
 'sell_5_vol': 368600.0,
 'sell_4': 11.94,
 'sell_4_vol': 202900.0,
 'sell_3': 11.93,
 'sell_3_vol': 132900.0,
 'sell_2': 11.92,
 'sell_2_vol': 123000.0,
 'sell_1': 11.91,
 'sell_1_vol': 154800.0,
 'buy_1': 11.9,
 'buy_1_vol': 514200.0,
 'buy_2': 11.89,
 'buy_2_vol': 51400.0,
 'buy_3': 11.88,
 'buy_3_vol': 155300.0,
 'buy_4': 11.87,
 'buy_4_vol': 105600.0,
 'buy_5': 11.86,
 'buy_5_vol': 58000.0,
 '最新': 11.91,
 '均价': 11.54,
 '涨幅': 0.0,
 '涨跌': 0.0,
 '总手': 1522392.0,
 '金额': 1756322367.0,
 '换手': 3.3,
 '量比': 3.72,
 '最高': 12.05,
 '最低': 11.11,
 '今开': 11.11,
 '昨收': 11.91,
 '涨停': 13.1,
 '跌停': 10.72,
 '外盘': 785580.0,
 '内盘': 736811.0}

In [5]:
import datetime

# 股票代码列表
symbols = ["stock_code1", "stock_code2", "stock_code3"]  # 替换为实际股票代码

def is_within_trading_hours(current_time):
    # 定义交易时间段
    morning_start = current_time.replace(hour=9, minute=15, second=0, microsecond=0)
    morning_end = current_time.replace(hour=11, minute=30, second=0, microsecond=0)
    afternoon_start = current_time.replace(hour=13, minute=0, second=0, microsecond=0)
    afternoon_end = current_time.replace(hour=15, minute=0, microsecond=0)
    
    return (morning_start <= current_time <= morning_end) or (afternoon_start <= current_time <= afternoon_end)

def fetch_and_store(symbol):
    # 获取当前时间
    current_time = datetime.now()
    print(current_time)

    if is_within_trading_hours(current_time):
        # 获取股票数据
        df = ak.stock_bid_ask_em(symbol=symbol)

        # 转换为字典
        data_dict = df.set_index('item').to_dict()['value']

        # 添加时间戳和股票代码到数据字典
        data_dict['timestamp'] = current_time
        data_dict['sym'] = symbol

        # 转换数据为 DataFrame
        transformed_df = pd.DataFrame([data_dict])

        # 将数据插入到 DolphinDB
        s.upload({"data": transformed_df})
        s.run("""
        append!(loadTable("dfs://Quote", "t"), data)
        """)

while True:
    # 使用多线程获取多个股票代码的数据
    with ThreadPoolExecutor(max_workers=len(symbols)) as executor:
        executor.map(fetch_and_store, symbols)

    # 每3秒检查一次
    time.sleep(3)

NameError: name 'ThreadPoolExecutor' is not defined

#### 2.2.1 集合竞价概念板块排名

In [8]:
import datetime
import os
import pandas as pd

#记录排名
# basename: 板块类型（概念板块、行业板块）
# start_time, end_time: 排名时间段
# sector: 板块名称
def recordRank(basename,start_time,end_time):
    now = datetime.datetime.now()
    tStr = end_time.strftime("%Y%m%d%H%M")
    csvfile=f'{basename}{tStr}.txt'
    if now.time() < end_time and now.time() >= start_time:
        if basename=='conceptRank':
            rank=ak.stock_board_concept_name_em()
        elif basename=='industryRank':
            rank=ak.stock_board_industry_name_em()
        rank.to_csv(csvfile)
    else:
        if os.path.exists(csvfile):
            rank=pd.read_csv(csvfile)
        else:
            print('非指定时间段，没有数据源.')
            rank=pd.DataFrame()
    if not rank.empty:
        mvtotal=rank['总市值'].sum()
        rank['市值比']=rank['总市值']/mvtotal
        rank['上涨家数%']=rank['上涨家数']/(rank['上涨家数']+rank['下跌家数'])
        columns=['排名','板块名称','板块代码','涨跌幅','市值比','换手率','上涨家数%','领涨股票','领涨股票-涨跌幅']
        rank=rank[columns]
    return rank

#排名靠前的板块的股票明细排名
# sectorRankDf: 板块排名DataFrame(recordRank生成)
# sectorNum,stockNum: 返回板块数、股票数
def top5sectorStocks(basename,start_time,end_time,sectorRankDf,sectorNum=5,stockNum=5):
    top5={}
    now = datetime.datetime.now()
    tStr = start_time.strftime("%Y%m%d%H%M")
    if sectorRankDf.empty:
        print('非指定时间段，没有数据源。')
    else:
        print('排名前5领涨板块成分股列表。')
        for sym in sectorRankDf['板块名称'].to_list()[:5]:
            csvfile=f'{basename}_{sym}{tStr}.txt'
            if now.time() < end_time and now.time() >= start_time:
                if basename=='conceptRank':
                    top5[sym]=ak.stock_board_concept_cons_em(sym)
                else:
                    top5[sym]=ak.stock_board_industry_cons_em(sym)
                top5[sym].to_csv(csvfile)
            else:
                if os.path.exists(csvfile):
                    top5[sym]=pd.read_csv(csvfile)
                else:
                    print('No record stored and not available from source.')
                    top5[sym]=pd.DataFrame()

            if not top5[sym].empty:
                atotal=top5[sym]['成交额'].sum()
                top5[sym]['成交额比']=top5[sym]['成交额']/atotal
                columns=['代码','名称','涨跌幅','成交额','换手率','市盈率-动态','市净率']
                top5[sym]=top5[sym][columns][:stockNum]
                print(f'板块：{sym}:\n{top5[sym]}\n\n')
    return top5   

In [9]:
#当天集合竞价阶段概念板块排名
basename='conceptRank'
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
rank0=recordRank(basename,start_time,end_time)
rank0.head(20)

非指定时间段，没有数据源.


""


In [10]:
top5_concept_stocks=top5sectorStocks(basename,start_time,end_time,rank0)

非指定时间段，没有数据源。


#### 2.2.3 开盘后15分钟概念板块排名

In [11]:
start_time = datetime.time(9, 31)
end_time = datetime.time(9, 52)
rank1=recordRank(basename,start_time,end_time)
rank1.head(20)

,排名,板块名称,板块代码,涨跌幅,市值比,换手率,上涨家数%,领涨股票,领涨股票-涨跌幅
0,1,BC电池,BK1148,3.51,0.000274,0.65,0.909091,广信材料,20.02
1,2,Chiplet概念,BK1101,3.28,0.000369,1.20,0.961538,苏州固锝,9.99
2,3,高带宽内存,BK1152,3.08,0.000090,1.15,1.000000,国芯科技,8.79
3,4,荣耀概念,BK1177,3.08,0.000616,0.69,0.980392,崇达技术,9.99
4,5,机器人执行器,BK1145,2.94,0.000233,0.79,1.000000,绿的谐波,4.91
5,6,MLCC,BK0890,2.89,0.000117,0.41,1.000000,洁美科技,6.60
6,7,汽车芯片,BK0969,2.86,0.000603,0.71,0.978723,希荻微,10.45
7,8,AI手机,BK1162,2.83,0.000467,0.84,0.939394,希荻微,10.45
8,9,激光雷达,BK1002,2.82,0.000228,0.68,0.935484,东田微,11.12
9,10,噪声防治,BK1082,2.79,0.000020,1.04,1.000000,德尔股份,13.35


排名前5概念板块领涨成分股

In [12]:
top5_concept_stocks=top5sectorStocks(basename,start_time,end_time,rank1)

排名前5领涨板块成分股列表。
板块：BC电池:
       代码    名称    涨跌幅           成交额    换手率  市盈率-动态   市净率
0  300537  广信材料  20.02  2.537559e+08   8.22   90.86  5.33
1  002079  苏州固锝   9.99  3.186958e+08   3.45  172.06  3.11
2  688700  东威科技   5.60  6.985421e+07   0.73  105.55  5.56
3  600732  爱旭股份   5.56  3.272845e+08   1.35   -7.44  4.60
4  001255  博菲电气   3.92  9.960661e+07  13.78  175.84  4.01


板块：Chiplet概念:
       代码    名称   涨跌幅           成交额   换手率  市盈率-动态   市净率
0  002079  苏州固锝  9.99  3.186958e+08  3.45  172.06  3.11
1  603005  晶方科技  5.27  7.137143e+08  3.86   76.76  4.47
2  002845   同兴达  5.06  7.852009e+07  1.96   54.96  1.93
3  603011  合锻智能  5.04  7.207263e+07  1.90  382.44  1.75
4  688603  天承科技  4.16  3.130320e+07  1.27   89.40  6.19


板块：高带宽内存:
       代码    名称   涨跌幅           成交额   换手率  市盈率-动态   市净率
0  688262  国芯科技  8.83  2.035809e+08  2.65  -58.34  4.40
1  603005  晶方科技  5.27  7.137143e+08  3.86   76.76  4.47
2  002559  亚威股份  3.28  3.929880e+07  0.88   36.91  2.80
3  688733   壹石通  3.04  1.020984e+07  0.2

#### 2.2.3 集合竞价行业板块涨跌幅

In [13]:
#当天集合竞价到开盘后30分钟涨跌幅排名
basename='industryRank'
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
rank3=recordRank(basename,start_time,end_time)
rank3.head(20)

非指定时间段，没有数据源.


""


行业板块中成分股

In [14]:
top5_industry_stocks=top5sectorStocks(basename,start_time,end_time,rank3)

非指定时间段，没有数据源。


#### 2.2.4 开盘15分钟行业板块涨跌幅

In [15]:
basename='industryRank'
start_time = datetime.time(9, 31)
end_time = datetime.time(9, 51)
rank4=recordRank(basename,start_time,end_time)
rank4.head(20)

,排名,板块名称,板块代码,涨跌幅,市值比,换手率,上涨家数%,领涨股票,领涨股票-涨跌幅
0,1,电子化学品,BK1039,3.42,0.002839,1.23,0.961538,广信材料,20.02
1,2,能源金属,BK1015,2.92,0.003675,0.65,1.000000,威领股份,9.98
2,3,半导体,BK1036,2.73,0.044129,0.85,0.980769,苏州固锝,9.99
3,4,光伏设备,BK1031,2.57,0.016223,0.44,0.986842,欧晶科技,10.00
4,5,光学光电子,BK1038,2.53,0.011068,0.66,0.930000,联得装备,11.94
5,6,电子元件,BK0459,2.35,0.018232,0.64,0.956835,远 望 谷,9.98
6,7,航天航空,BK0480,2.26,0.009893,0.39,1.000000,航天宏图,8.23
7,8,电机,BK1030,2.19,0.001868,0.80,0.923077,兆威机电,5.71
8,9,消费电子,BK1037,2.10,0.020011,0.77,0.900000,朝阳科技,9.99
9,10,汽车零部件,BK0481,1.91,0.016917,1.14,0.885845,长春一东,10.02


行业板块中成分股

### 2.2.5 集合竞价阶段涨停个股热点明细

In [16]:
#在指定时段获取涨停数据，若时间不在指定时段，则读取历史最后一次下载的记录。
def stockLimitUp(start_time,end_time):
    now = datetime.datetime.now()
    tStr = end_time.strftime("%Y%m%d%H%M")
    csvfile=f'limitUp{tStr}.txt'
    if now.time() < end_time and now.time() >= start_time:
        rank=ak.stock_zh_a_spot_em()
        rank.to_csv(csvfile)
    else:
        if os.path.exists(csvfile):
            rank=pd.read_csv(csvfile)
        else:
            print('No record stored and not available from source.')
            rank=pd.DataFrame()
    if not rank.empty:
        columns=['代码','名称','涨跌幅','量比','换手率','市盈率-动态','市净率','流通市值','涨速','5分钟涨跌','60日涨跌幅','年初至今涨跌幅']
        rank=rank[columns].sort_values(by='涨跌幅', ascending=False)
    return rank

In [17]:
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
openA = stockLimitUp(start_time,end_time)
openA.head(50)

No record stored and not available from source.


""


### 2.2.6 开盘15分钟涨停个股热点明细


In [18]:
start_time = datetime.time(9, 31)
end_time = datetime.time(9, 53)
open15 = stockLimitUp(start_time,end_time)
open15.head(50)

,代码,名称,涨跌幅,量比,换手率,市盈率-动态,市净率,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
0,920060,N万源通,334.59,NaN,48.77,57.39,7.53,1.438030e+09,1.02,6.24,334.59,334.59
1,831832,科达自控,28.90,14.75,10.78,79.26,3.70,2.055155e+09,3.83,6.99,204.89,87.89
2,300537,广信材料,20.02,13.33,8.22,90.86,5.33,3.132526e+09,0.00,0.00,42.50,10.91
3,300607,拓斯达,14.20,7.24,14.99,908.95,4.01,7.564066e+09,3.07,4.40,109.43,54.89
4,300473,德尔股份,13.35,27.86,11.23,113.48,2.35,3.628790e+09,2.15,2.46,51.82,32.39
5,688068,热景生物,12.09,15.10,5.08,-67.89,1.13,3.678643e+09,1.09,0.53,40.07,-2.62
6,300545,联得装备,11.91,13.69,6.33,25.41,3.58,4.169197e+09,-0.64,-1.41,61.94,7.62
7,301183,东田微,11.12,11.47,10.39,83.84,5.35,2.587380e+09,-1.47,-2.70,51.04,30.74
8,688173,希荻微,10.59,46.99,8.08,-24.66,4.06,3.737210e+09,-0.45,0.26,64.84,-10.97
9,002344,海宁皮城,10.08,5.63,0.35,69.89,0.80,6.714798e+09,0.00,0.00,67.41,29.06


#### 2.2.7 板块动态评析

本轮行情启动股票热点追踪（需要从历史行情中组合分析，现有股票涨停池昨日和今日都不稳定）

In [19]:
from datetime import datetime, timedelta

def get_stock_zt_pool(start_date, end_date):
    current_date = datetime.strptime(start_date, '%Y%m%d')
    end_date = datetime.strptime(end_date, '%Y%m%d')
    
    concat_zt_df = pd.DataFrame()

    while current_date <= end_date:
        date_str = current_date.strftime('%Y%m%d')
        try:
            stock_zt_pool_strong_em_df = ak.stock_zt_pool_previous_em(date=date_str)
            stock_zt_pool_strong_em_df['日期'] = pd.to_datetime(date_str)
            concat_zt_df = pd.concat([concat_zt_df, stock_zt_pool_strong_em_df], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
        
        current_date += timedelta(days=1)
    
    return concat_zt_df

In [20]:
syclestart='20240925'
#ak.stock_zt_pool_strong_em(date=syclestart)

In [21]:
#本轮行情启动日
syclestart='20240924'
result_df = get_stock_zt_pool(syclestart, lastday)
result_df

NameError: name 'lastday' is not defined

## 4.个股市场


### 4.1 跟进股票列表
#### 4.1.1 历史K线

In [ ]:
def plotDcStockSingle(df,days):
    OCHLVDict={'日期':'Date','开盘':'Open','收盘':'Close','最高':'High','最低':'Low','成交量':'Volume'}
    print(f'{days}K线图')
    plotStock(df.tail(days),OCHLVDict)

In [ ]:
#根据代码获取股票信息
#symbol:若是字符串，则返回单个股信息；若是列表，则遍历元素，返回相应列表。
def getStockInfo(symbol):
    if type(symbol).__name__=='str':
        stocknamedf=ak.stock_individual_info_em(symbol=symbol)
        stockname=stocknamedf.loc[stocknamedf['item']=='股票简称','value'][1]
        stockIPO=stocknamedf.loc[stocknamedf['item']=='上市时间','value'][7]
        stockIndustry=stocknamedf.loc[stocknamedf['item']=='行业','value'][6]
        stockTAmount=stocknamedf.loc[stocknamedf['item']=='总股本','value'][2]
        stockFAmount=stocknamedf.loc[stocknamedf['item']=='流通股','value'][3]
        stockTValue=stocknamedf.loc[stocknamedf['item']=='总市值','value'][4]
        stockFValue=stocknamedf.loc[stocknamedf['item']=='流通市值','value'][5]
    else:
        stockname,stockIPO,stockIndustry,stockTAmount,stockFAmount,stockTValue,stockFValue=[],[],[],[],[],[],[]
        for sym in symbol:
            stocknamedf=ak.stock_individual_info_em(symbol=sym)
            stockname.append(stocknamedf.loc[stocknamedf['item']=='股票简称','value'][1])
            stockIPO.append(stocknamedf.loc[stocknamedf['item']=='上市时间','value'][7])
            stockIndustry.append(stocknamedf.loc[stocknamedf['item']=='行业','value'][6])
            stockTAmount.append(stocknamedf.loc[stocknamedf['item']=='总股本','value'][2])
            stockFAmount.append(stocknamedf.loc[stocknamedf['item']=='流通股','value'][3])
            stockTValue.append(stocknamedf.loc[stocknamedf['item']=='总市值','value'][4])
            stockFValue.append(stocknamedf.loc[stocknamedf['item']=='流通市值','value'][5])
            
    return {'名称':stockname,'IPO':stockIPO,'行业':stockIndustry,'总股本':stockTAmount,'流通股':stockFAmount,'总市值':stockTValue,'流通市值':stockFValue}

#画指定周期的股票烛线图
def plotDcStocks(stocksymb,periods):
    stockDict={}
    for p in periods:
        stockdf = ak.stock_zh_a_hist(symbol=stocksymb,period=p)
        stockname=getStockInfo(stocksymb)['名称']
        stockDict[p]=stockdf
        print(f'{stockname}-周期：{p}\n{stockdf}')
        plotDcStockSingle(stockdf,100)
    stockDict['name']=stockname
    return stockDict

In [ ]:
stocksymb='300339'
periods=['daily','weekly','monthly']
stockDict=plotDcStocks(stocksymb,periods)

### 4.1.2 盘口信息汇总
#### 4.1.2.1 成交和挂单明细

获取腾讯当日历史分笔数据

In [ ]:
# 根据code代码返回“交易所+代码” 信息，或者反过来
def transferExSym(code):
    if len(code)>6:
        rescode=code[-6:]
    else:
        source={}
        source['sh']=ak.stock_sh_a_spot_em()['代码'].to_list()
        source['sz']=ak.stock_sz_a_spot_em()['代码'].to_list()
        source['bj']=ak.stock_bj_a_spot_em()['代码'].to_list()
        for k in source.keys():
            if code in source[k]:
                rescode=k+code
                break
    return rescode

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#区分北交所和沪深股市，刻画市场深度。按照涨跌停板，作为极值，按照价格涨幅的measure度量区间
def priceMarketDepth(df,measure=0.01):

    open_price = df.loc[df['成交时间'] == '09:30:00', '成交价格'].iloc[0]

    # 设置涨跌停板系数
    if stocksymb == 'bj':
        limit_factor = 0.2
    else:
        limit_factor = 0.1

    # 计算最高价和最低价
    high_price = open_price * (1 + limit_factor)
    low_price = open_price * (1 - limit_factor)

    # 定义价格区间，每1%一档
    price_bins = np.round(np.arange(low_price, high_price, (high_price - low_price) * measure),2)

    # 将价格按照区间划分
    df['价格区间'] = pd.cut(df['成交价格'], bins=price_bins, right=False)

    # 按照价格区间汇总成交量
    volume_summary = df.groupby('价格区间',observed=False)['成交量'].sum().reset_index()

    # 新建 df['prop'] 列
    df['prop'] = df['性质'].apply(lambda x: -1 if x == '卖盘' else (1 if x == '买盘' else 0))
    
    # 按照价格区间汇总成交量和 prop 之和
    summary = df.groupby('价格区间',observed=False).agg({
        '成交量': 'sum',
        'prop': 'sum'
    }).reset_index()
    
    # 提取价格区间的中点和边界
    summary['价格'] = summary['价格区间'].apply(lambda x: round(x.mid,2))
    summary['起始价格'] = summary['价格区间'].apply(lambda x: round(x.left,2))
    summary['终止价格'] = summary['价格区间'].apply(lambda x: round(x.right,2))

    # 分离买盘和卖盘数据
    summary['买盘深度'] = summary['prop'].apply(lambda x: x if x > 0 else 0)
    summary['卖盘深度'] = -summary['prop'].apply(lambda x: -x if x < 0 else 0)

    # 绘制市场深度水平条形图
    fig, ax = plt.subplots(figsize=(14, 21))

    # 绘制买卖盘深度图
    ax.barh(summary['价格'], summary['买盘深度'], color='green', label='buy')
    ax.barh(summary['价格'], summary['卖盘深度'], color='red', label='sell')
    
    # 在图上标注区间价格的起始和终止价格
    for index, row in summary.iterrows():
        ax.text(row['买盘深度'] + max(summary['买盘深度'].max(), summary['卖盘深度'].max())*measure, row['价格'], f'{row["起始价格"]:.2f}', va='center', ha='left', fontsize=8, color='black')
        ax.text(-row['卖盘深度'] - max(summary['买盘深度'].max(), summary['卖盘深度'].max())*measure, row['价格'], f'{row["终止价格"]:.2f}', va='center', ha='right', fontsize=8, color='black')

    # 设置图表标题和标签
    ax.set_title('Market Depth')
    ax.set_xlabel('Depth(Amount)')
    ax.set_ylabel('Price')
    ax.legend()

    
    # 显示图表
    plt.show()
    
    return summary

In [ ]:
from rich.console import Console
from rich.table import Table

#美观显示DataFrame表格
class DataFramePretty(object):
    def __init__(self, df: pd.DataFrame, min_max_cols=[]) -> None:
        self.data = df
        self.min_max_cols = min_max_cols

    def __set_min_max_style(self, df: pd.DataFrame):
        imax = self.data.idxmax()
        imin = self.data.idxmin()

        max_tmpl = "[b on red3]{}[/b on red3]"
        min_tmpl = "[i on green3]{}[/i on green3]"

        for col in self.min_max_cols:
            col_idx = imax.index.tolist().index(col)
            val = df.iloc[imax[col], col_idx]
            df.iloc[imax[col], col_idx] = max_tmpl.format(val)

            col_idx = imin.index.tolist().index(col)
            val = df.iloc[imin[col], col_idx]
            df.iloc[imin[col], col_idx] = min_tmpl.format(val)

    def show(self):
        table = Table(
            title="DataFrame",
            title_style="i on dark_cyan",
            header_style="bold cyan",
        )

        # self.data是原始数据
        # df 是用来显示的数据
        df = self.data.copy()
        for col in df.columns:
            df[col] = df[col].astype("str")
            table.add_column(col)

        self.__set_min_max_style(df)

        for idx in range(len(df)):
            table.add_row(*df.iloc[idx].tolist())

        console = Console()
        console.print(table)

In [ ]:
#剔除中性单以后，列出活跃买卖市场深度
def plotActiveMarketDepth(stocksymbol):
    
    print(f'市场深度：{stocksymb}')
    #一次性获得历史分笔数据（当天即当天之前的交易时间；隔天即昨天）
    yesterdayTicks=ak.stock_zh_a_tick_tx_js(symbol=transferExSym(stocksymb))
    
    md=priceMarketDepth(yesterdayTicks)
    md = md[(md['买盘深度'] != 0) | (md['卖盘深度'] != 0)]
    mdp=DataFramePretty(md)
    mdp.show()
    return  md

剔除中性盘以后，主动买卖盘的市场深度

In [ ]:
plotActiveMarketDepth(stocksymb)

### 4.1.3 个股追踪
追踪股票列表

In [ ]:
focusStocks=['300576','600839','600157','000717','002607','600127','600605','600217','300727','301421','002843','600968','300159','601688']
getInfo=getStockInfo(focusStocks)
Analist=pd.DataFrame({'代码':focusStocks,'名称':getInfo['名称'],'IPO':getInfo['IPO'],'行业':getInfo['行业'],'流通股':getInfo['流通股'],'总股本':getInfo['总股本']})
Analist['流通股']=Analist['流通股'].astype(int)
Analist['总股本']=Analist['总股本'].astype(int)
Analist['流通率']=Analist['流通股']/Analist['总股本']
Analist

时序数据库准备

In [ ]:
import dolphindb as ddb
from concurrent.futures import ThreadPoolExecutor
import dolphindb.settings as keys

s = ddb.session("localhost", 8848, "admin", "123456")
s.run("quote_tab = table(1:0, `timestamp`sym`sell_5`sell_5_vol`sell_4`sell_4_vol`sell_3`sell_3_vol`sell_2`sell_2_vol`sell_1`sell_1_vol`buy_1`buy_1_vol`buy_2`buy_2_vol`buy_3`buy_3_vol`buy_4`buy_4_vol`buy_5`buy_5_vol`最新`均价`涨幅`涨跌`总手`金额`换手`量比`最高`最低`今开`昨收`涨停`跌停`外盘`内盘, [TIMESTAMP, SYMBOL, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE,DOUBLE,DOUBLE])")
quote_tab = s.table(data="quote_tab")
dbPath="dfs://quotes"

if not s.existsDatabase(dbPath):
    db = s.database(dbPath=dbPath, partitionType=keys.LIST, partitions=focusStocks,engine='TSDB')
    db.createTable(quote_tab, "quotes", ["sym"])
    
qt = s.table(dbPath=dbPath, data="quotes")


In [ ]:
stockCharts,marketDepths={},{}

for f in focusStocks:
    print('='*75)
    stockCharts[f]=plotDcStocks(f,periods)
    stockname=stockCharts[f]['name']
    print(f'{stockname}（{f}）的市场深度：')
    marketDepths[f]=plotActiveMarketDepth(f)
    

In [ ]:
stock_bid_ask_em_df = ak.stock_bid_ask_em(symbol="600839")
stock_bid_ask_em_df

In [ ]:
import akshare as ak
import dolphindb as ddb
import pandas as pd
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# 初始化 DolphinDB 连接
s = ddb.session()
s.connect("localhost", 8848, "admin", "123456")  # 替换为实际的主机、端口、用户名和密码

# 创建 DolphinDB 表
s.run("""
if (existsDatabase("dfs://Quote")) {
    dropDatabase("dfs://Quote")
}
t = table(1:0, `timestamp`sym`sell_5`sell_5_vol`sell_4`sell_4_vol`sell_3`sell_3_vol`sell_2`sell_2_vol`sell_1`sell_1_vol`buy_1`buy_1_vol`buy_2`buy_2_vol`buy_3`buy_3_vol`buy_4`buy_4_vol`buy_5`buy_5_vol`最新`均价`涨幅`涨跌`总手`金额`换手`量比`最高`最低`今开`昨收`涨停`跌停`外盘`内盘, [TIMESTAMP, SYMBOL, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE])
database("dfs://Quote", VALUE, t)
""")

def is_within_trading_hours(current_time):
    # 定义交易时间段
    morning_start = current_time.replace(hour=9, minute=15, second=0, microsecond=0)
    morning_end = current_time.replace(hour=11, minute=30, second=0, microsecond=0)
    afternoon_start = current_time.replace(hour=13, minute=0, second=0, microsecond=0)
    afternoon_end = current_time.replace(hour=15, minute=0, microsecond=0)
    
    return (morning_start <= current_time <= morning_end) or (afternoon_start <= current_time <= afternoon_end)

def fetch_and_store(symbol):
    # 获取当前时间
    current_time = datetime.now()

    if is_within_trading_hours(current_time):fgggttryy
        # 获取股票数据
        df = ak.stock_bid_ask_em(symbol=symbol)

        # 转换为字典
        data_dict = df.set_index('item').to_dict()['value']

        # 添加时间戳和股票代码到数据字典
        data_dict['timestamp'] = current_time
        data_dict['sym'] = symbol

        # 转换数据为 DataFrame
        transformed_df = pd.DataFrame([data_dict])

        # 将数据插入到 DolphinDB
        s.upload({"data": transformed_df})
        s.run("""
        append!(loadTable("dfs://Quote", "t"), data)
        """)]]
        ]

while True:
    
    # 使用多线程获取多个股票代码的数据
    with ThreadPoolExecutor(max_workers=len(focusStocks)) as executor:
        executor.map(fetch_and_store, focusStocks)

    # 每3秒检查一次
    time.sleep(3)
